In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Load the data
def load_data(file_path, test=False):
    if test:
        data = pd.read_csv(file_path, delimiter=' ::: ', engine='python', names=['ID', 'TITLE', 'DESCRIPTION'])
    else:
        data = pd.read_csv(file_path, delimiter=' ::: ', engine='python', names=['ID', 'TITLE', 'GENRE', 'DESCRIPTION'])
    return data

# Preprocess the data
def preprocess_data(data):
    data['DESCRIPTION'] = data['DESCRIPTION'].str.lower()
    return data

# Converting text to features using TF-IDF
def text_to_features(description, vectorizer=None, fit=False):
    if fit:
        vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
        X = vectorizer.fit_transform(description)
    else:
        X = vectorizer.transform(description)
    return X, vectorizer

# Train the model
def train_model(X, y, model_type):
    if model_type == 'logistic_regression':
        model = LogisticRegression(max_iter=1000)
    elif model_type == 'naive_bayes':
        model = MultinomialNB()
    model.fit(X, y)
    return model

# Evaluate the model
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Predicting Genre for Test Data
def predict_genre(model, X_test):
    predictions = model.predict(X_test)
    return predictions


Loading and preprocessing the Training Data

In [2]:
train_file_path = './data/train_data.txt'

train_data = load_data(train_file_path)
train_data = preprocess_data(train_data)
train_data.head()


,ID,TITLE,GENRE,DESCRIPTION
0,1,Oscar et la dame rose (2009),drama,listening in to a conversation between his doc...
1,2,Cupid (1997),thriller,a brother and sister with a past incestuous re...
2,3,"Young, Wild and Wonderful (1980)",adult,as the bus empties the students for their fiel...
3,4,The Secret Sin (1915),drama,to help their unemployed father make ends meet...
4,5,The Unrecovered (2007),drama,the film's title refers not only to the un-rec...


Converting training text to feeatures

In [3]:
X_train, vectorizer = text_to_features(train_data['DESCRIPTION'], fit=True)
y_train = train_data['GENRE']

Split training data for evaluation

In [4]:
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

Train and evaluate Logistic Regression

In [5]:
lr_model = train_model(X_train_split, y_train_split, 'logistic_regression')
lr_accuracy = evaluate_model(lr_model, X_val_split, y_val_split)
print("Logistic Regression - Accuracy:", lr_accuracy)

Logistic Regression - Accuracy: 0.5794521811306834


Train and evaluate Naive Bayes

In [6]:
nb_model = train_model(X_train_split, y_train_split, 'naive_bayes')
nb_accuracy = evaluate_model(nb_model, X_val_split, y_val_split)
print("Naive Bayes - Accuracy:", nb_accuracy)

Naive Bayes - Accuracy: 0.52310246241815


Loading and preprocessing Test Data

In [7]:
test_file_path = './data/test_data.txt'

test_data = load_data(test_file_path, test=True)
test_data = preprocess_data(test_data)
test_data.head()

,ID,TITLE,DESCRIPTION
0,1,Edgar's Lunch (1998),"l.r. brane loves his life - his car, his apart..."
1,2,La guerra de papá (1977),"spain, march 1964: quico is a very naughty chi..."
2,3,Off the Beaten Track (2010),one year in the life of albin and his family o...
3,4,Meu Amigo Hindu (2015),"his father has died, he hasn't spoken with his..."
4,5,Er nu zhai (1955),before he was known internationally as a marti...


Converting test text to features

In [8]:
X_test, _ = text_to_features(test_data['DESCRIPTION'], vectorizer=vectorizer, fit=False)

Predicting genre using the best model (In this case Logistic Regression)

In [9]:
predictions = predict_genre(lr_model, X_test)
result = pd.DataFrame({'ID': test_data['ID'], 'TITLE': test_data['TITLE'],'PREDICTED GENRE': predictions})
print(result)

          ID                           TITLE PREDICTED GENRE
0          1            Edgar's Lunch (1998)           short
1          2        La guerra de papá (1977)           drama
2          3     Off the Beaten Track (2010)     documentary
3          4          Meu Amigo Hindu (2015)           drama
4          5               Er nu zhai (1955)           drama
...      ...                             ...             ...
54195  54196  "Tales of Light & Dark" (2013)           drama
54196  54197     Der letzte Mohikaner (1965)          action
54197  54198             Oliver Twink (2007)          comedy
54198  54199               Slipstream (1973)           drama
54199  54200       Curitiba Zero Grau (2010)     documentary

[54200 rows x 3 columns]


Saving result to a file

In [10]:
result.to_csv('predicted_genres.csv', index=False, sep='-')